# PLOTS AND OTHER FUNCTIONS

In [ ]:
# default_exp plots

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import numpy as np
from mpmath import polylog, e, lerchphi
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
#export
def plot_general(xlabel, ylabel, title, figsize):
        
    """General plot characteristics"""
    
    fig = plt.figure(figsize = figsize)
    ax = fig.add_subplot(111)
    plt.rc('xtick',labelsize=40)
    plt.rc('ytick',labelsize=40)
    plt.ylabel(ylabel, fontsize=40)
    plt.xlabel(xlabel, fontsize=40)
    plt.grid(axis='both')
    plt.title(title , fontsize = 40)
    
    return fig, ax


def plot_IPR(x, IPR_map, evals_total, path, name_file, xlabel=r'$\mu$', title='IPR', normalize = False, set_ylim = False, ylim = [-2,2]):
    
    """Plot bandstructure with colormap of IPR"""
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    plt.rc('xtick',labelsize=40)
    plt.rc('ytick',labelsize=40)
    plt.ylabel('Energy(eV)', fontsize=40)
    plt.xlabel(xlabel, fontsize=40)
    plt.grid(axis='both')
    plt.title(title, fontsize = 40)

    for i in range(len(evals_total.T)):   

        y = np.real(evals_total.T[i])
        IPR = IPR_map[i]

        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        
        if normalize == False:
            norm = plt.Normalize(IPR_map.min(), IPR_map.max())
        else:
            norm = plt.Normalize(0,1)
        lc = LineCollection(segments, cmap='viridis', norm=norm)
        lc.set_array(IPR)
        lc.set_linewidth(2)
        line = ax.add_collection(lc)

    fig.colorbar(line, ax=ax)
    ax.set_xlim(x.min(), x.max())
    if set_ylim == False:
        ax.set_ylim(np.real(evals_total).min(), np.real(evals_total).max())
    else:
        ax.set_ylim(ylim[0], ylim[1])

    plt.savefig(path+name_file)
    
    return fig, ax


def plot_av(mu_list, IPR, NPR, path, name_file, xlabel=r'$\mu$', title='IPR'):
    
    """Plot av NPR and IPR with colormap of IPR"""
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    plt.rc('xtick',labelsize=40)
    plt.rc('ytick',labelsize=40)
    plt.xlabel(xlabel, fontsize=40)
    plt.grid(axis='both')
    plt.title(title, fontsize = 40)
    plt.plot(mu_list, IPR, 'bo-', label='IPR')
    plt.plot(mu_list, NPR, 'ro-', label='NPR')    
    plt.legend(fontsize=40)
    plt.savefig(path+name_file)
    
    return fig, ax


class HistogramNormalize(colors.Normalize):
    
    '''Set normalized color map'''

    def __init__(self, data, vmin=None, vmax=None):
        if vmin is not None:
            data = data[data > vmin]
        if vmax is not None:
            data = data[data < vmax]

        sorted_data = np.sort(data.flatten())
        self.sorted_data = sorted_data[np.isfinite(sorted_data)]
        colors.Normalize.__init__(self, vmin, vmax)

    def __call__(self, value, clip=None):
        return np.ma.masked_array(np.searchsorted(self.sorted_data, value) /
                                  len(self.sorted_data))


def plot_wf(evecs, evals, IPR, title, path, name_file):
    
    """Plot evecs to see the localization"""
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    plt.rc('xtick',labelsize=40)
    plt.rc('ytick',labelsize=40)
    plt.xlabel('sites', fontsize=40)
    plt.ylabel(r'$\mid\psi\mid²$', fontsize=40)
    plt.grid(axis='both')
    plt.title(title, fontsize = 40)
    for i in range(len(evecs)):
        plt.plot(range(len(evecs[i])), np.abs(evecs[i])**2, 'o-', label='E = %.2f, IPR= %.2f' %(evals[i], IPR[i]), alpha=0.6)

    plt.legend(fontsize=40)
    plt.savefig(path+name_file)
    
    return